This notebook quantifies chances in single unit activity, MUA, theta, and SWRs through the course of ANI administration in a male rat.

In [ ]:
# Import all the functions you need
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from pathlib import Path

# Comment here to add in for either mac or linux computer
# sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/NeuroPy'])  # For mac laptop
sys.path.extend(['/data/GitHub/NeuroPy'])  # for linux

from neuropy import core
from neuropy.io import (optitrackio,
                        dlcio,
                        )
from neuropy.io.neuroscopeio import NeuroscopeIO
from neuropy.io.binarysignalio import BinarysignalIO 
from neuropy.io.miniscopeio import MiniscopeIO
from neuropy.core import Epoch

plt.rcParams['pdf.fonttype'] = 42


In [ ]:
# Define a class for a typical recording or set of recordings
class ProcessData:
    def __init__(self, basepath):
        basepath = Path(basepath)
        self.basepath = basepath
        xml_files = sorted(basepath.glob("*.xml"))
        assert len(xml_files) == 1, "Found more than one .xml file"
        
        fp = xml_files[0].with_suffix("")
        self.filePrefix = fp
        
        self.recinfo = NeuroscopeIO(xml_files[0])
        eegfiles = sorted(basepath.glob('*.eeg'))
        assert len(eegfiles) == 1, "Fewer/more than one .eeg file detected"
        self.eegfile = BinarysignalIO(eegfiles[0], n_channels=self.recinfo.n_channels,
                                     sampling_rate=self.recinfo.eeg_sampling_rate,
                                     )
        try:
            self.datfile = BinarysignalIO(eegfiles[0].with_suffix('.dat'),
                                         n_channels=self.recinfo.n_channels,
                                         sampling_rate=self.recinfo.dat_sampling_rate,
                                         )
        except FileNotFoundError:
            print('No dat file found, not loading')
                
        
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.recinfo.source_file.name})"
    
def Wedge_saline1():
    basepath = '/data3/Anisomycin/Recording_Rats/Wedge/2022_12_12_saline1' # for linux desktop
    return ProcessData(basepath)

def Wedge_ani1():
    basepath = '/data3/Anisomycin/Recording_Rats/Wedge/2022_12_13_anisomycin50mg' # for linux desktop
    return ProcessData(basepath)

def Wedge_saline2():
    basepath = '/data3/Anisomycin/Recording_Rats/Wedge/2022_12_14_saline2' # for linux desktop
    return ProcessData(basepath)
    
def Wedge_saline3():
    basepath = '/data3/Anisomycin/Recording_Rats/Wedge/2022_12_21_saline3' # for linux desktop
    return ProcessData(basepath)

Here are some quick functions I wrote that probably are better done in NeuroPy.

In [ ]:
def load_data(dir_use):
    clu_id = np.load(dir_use / 'spike_clusters.npy')
    amp = np.load(dir_use / 'amplitudes.npy')
    spike_times = np.load(dir_use / 'spike_times.npy')
    
    return clu_id, amp, spike_times

def parse_data(spike_times, amp, clu_id, good_units):
    SR = 30000
    for unit in good_units:
        unit_bool = clu_id == unit
        clu_times.append(spike_times[unit_bool] / SR)
        clu_amps.append(amp[unit_bool])

        return clu_times, clu_amps
    
def plot_FR(clu_times, spike_times, inj_time_sec, plot_name=""):
    bin_size = 10  # sec
    bins_use = np.arange(0, spike_times[-1] / 30000, bin_size)
    fig, asp = plt.subplots(len(good_units), 1, figsize=(20, 1.95 * len(good_units)), sharex=True)
    fig.suptitle(plot_name)
    for ida, (a, clu_t) in enumerate(zip(asp, clu_times)):
        sp_rate = []
        for bin_start, bin_end in zip(bins_use[:-1], bins_use[1:]):
            sp_rate.append(np.bitwise_and(clu_t > bin_start, clu_t < bin_end).sum() / bin_size)
        sp_rate = np.array(sp_rate)
        a.plot(bins_use[:-1] / 60, sp_rate)
        a.set_title(f'Good cell # {ida}')
        a.set_ylabel(f'Rate (Hz): {bin_size} sec bin')
        a.set_xlabel('time (min)')
        a.axvline(inj_time_sec / 60, color='g')
        

In [ ]:
def bin_MUA(spike_times, SR=30000, bin_size=10):
    bins_use = np.arange(0, spike_times[-1] / SR, bin_size)
    mua_rate = []
    for bin_start, bin_end in zip(bins_use[:-1], bins_use[1:]):
        mua_rate.append(np.bitwise_and(spike_times / SR > bin_start, spike_times / SR < bin_end).sum() / bin_size)
    
    mua_times = bins_use[:-1] / 60
    return mua_rate, mua_times


Below here are code scraps for posterity.

In [ ]:
dir_use = Path('/data3/Anisomycin/Recording_Rats/Wedge/2022_12_14_saline2/spyk-circ/Wedge_saline2/Wedge_saline2.GUI')
good_units = [4, 5, 6, 9, 11]
clu_id, amp, spike_times = load_data(dir_use)
clu_times, clu_amps = parse_data(spike_times, amp, clu_id, good_units)

In [ ]:
dir_use = Path('/data3/Anisomycin/Recording_Rats/Wedge/2022_12_12_saline/spyk-circ/Wedge_saline1/Wedge_saline1.GUI')
clu_id, amp, spike_times = load_data(dir_use)
good_units = [31, 41, 47, 53, 60, 61, 71, 74]
clu_times, clu_amps = parse_data(spike_times, amp, clu_id, good_units)


In [ ]:
dir_use = Path('/data3/Anisomycin/Recording_Rats/Wedge/2022_12_13_anisomycin50mg/spyk-circ/Wedge_ani1/Wedge_ani1.GUI')
good_units = [37, 45, 57, 66, 79, 94, 97]
clu_id, amp, spike_times = load_data(dir_use)
clu_times, clu_amps = parse_data(spike_times, amp, clu_id, good_units)
plot_FR(clu_times, spike_times, 16*60, plot_name="Wedge ANI1 (50mg)")
plt.gca().set_xlim([-5, 125])

In [ ]:
mua_rate_ani, mua_times_ani = bin_MUA(spike_times)
ax.plot(mua_times_ani, mua_rate_ani)
ax.set_title('ANI')